### Goal: Can we see subsets of dendritic cells in P7 multiome data


In [ ]:
import numpy as np
import pandas as pd
import os
import scanpy as sc
import scanpy.external as sce
import sys
import muon as mu
import muon.atac as ac
import matplotlib.pyplot as plt
import seaborn as sns
figures = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/pilot/230720_cristina_endo_questions'
sc_file = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/single_cell_files/share'
atac_dir = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/figures/atac'
rna_dir = '/home/carsten/alvira_bioinformatics/postnatal_lung_multiome/data/figures/rna'
os.makedirs(figures, exist_ok=True)
sc.set_figure_params(dpi=300, format="png")
sc.settings.figdir = figures


#### Load in multiomic data

In [ ]:
rna = sc.read(f'{sc_file}/p7_multiome_rna_processed.gz.h5ad')
atac = sc.read(f'{sc_file}/p7_multiome_atac_processed.gz.h5ad')
tf = sc.read(f'{sc_file}/p7_multiome_tf_processed.gz.h5ad')

In [ ]:
rna.X = rna.layers['soupx'].copy()
sc.pp.normalize_total(rna,target_sum=1e4)
sc.pp.log1p(rna,base=10)

In [ ]:
atac.X = atac.layers['counts'].copy()
sc.pp.normalize_total(atac,target_sum=1e4)
sc.pp.log1p(atac,base=10)

In [ ]:
tf.X = tf.layers['raw'].copy()
sc.pp.normalize_total(tf,target_sum=1e4)
sc.pp.log1p(tf,base=10)

#### Subset DC

In [ ]:
markers = {
    'pan':['Zbtb46', 'Flt3'],
    'cDC1':['Itgae','Irf8','Cadm1'],
    'cDC2':['Itgam', 'Sirpa', 'Irf4'],
    'mig-DC':['Mreg','Cacnb3','Fscn1','Ccl5','Ccr7',]
}
marker_ls = []
for x in markers.keys():
    marker_ls+=markers[x]

In [ ]:
dc = rna[rna.obs['celltype']=='DC'].copy()
sc.pp.highly_variable_genes(dc, n_top_genes=500, batch_key="mouse")
sc.pp.pca(dc, use_highly_variable=True)
sce.pp.harmony_integrate(dc, key='mouse', max_iter_harmony=20)
sc.pp.neighbors(dc, use_rep='X_pca')
sc.tl.leiden(
    dc,
    key_added=f"leiden_dc",
    resolution=0.9
)
sc.tl.umap(dc, min_dist=0.1)
sc.pl.umap(dc, color=marker_ls)
sc.pl.umap(dc, color=['leiden_dc','mouse'])


In [ ]:
dc.obs.groupby('leiden_dc')['mouse'].value_counts()

In [ ]:
sc.tl.rank_genes_groups(dc, "leiden_dc", method="wilcoxon")
sc.pl.rank_genes_groups_dotplot(
    dc,
    groupby="leiden_dc",
    dendrogram=False,
    n_genes=int(50 / len(dc.obs["leiden_dc"].unique())),
)
sc.pl.dotplot(dc,markers, groupby='leiden_dc')

In [ ]:
dc.obs.groupby('treatment')['leiden_dc'].value_counts(normalize=True)